In [ ]:
import open3d as o3d
import os
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
import math
import json
import random
import uuid
import ifcopenshell

from src.visualisation import *

create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)


### Dataset Creation

In [ ]:
# combine clouds
data_path = "/mnt/f/datasets/export/export/"
max_points = 4096

In [ ]:
classes = os.listdir(data_path)
print(classes)

In [ ]:
all_classes = []
element_count = 0
error_count = 0
for i, cl in enumerate(classes):
    all_elements = []
    class_path = data_path + cl
    elements = os.listdir(class_path)
    for j, el in tqdm(enumerate(elements)):
        try:
            element = np.loadtxt(class_path + '/' + el)

            # downsample
            if (len(element) > 0 and element.ndim == 2 and element.shape[1] == 4):
                if len(element)  > max_points:
                    #idx = np.random.randint(element.shape[0], size=max_points)
                    #element = element[idx :]
                    element = np.random.permutation(element)[:max_points]

                element = np.delete(element, 3, axis=1) # remove point index
                element = np.insert(element, 3, values=[element_count], axis=1) # add element index
                #print(element.shape)
                element_count += 1
                all_elements.append(element)
        except Exception as E:
            error_count += 1
            
            
    all_elements = np.vstack(all_elements)
    all_elements = np.insert(all_elements, 4, values=[i], axis=1) # add class index
    all_classes.append(all_elements)
    
all_classes = np.concatenate(all_classes)
print(all_classes.shape)
print ("errors: ", error_count)
        
        #print(points[0])

In [ ]:
pcd = o3d.t.geometry.PointCloud()
points = all_classes[:,0:3]
el_index  = [[i] for i in all_classes[:,3]]
cl_index  = [[i] for i in all_classes[:,4]]


pcd.point["positions"] = o3d.core.Tensor(points)
pcd.point["elements"] = o3d.core.Tensor(el_index)
pcd.point["classes"] = o3d.core.Tensor(cl_index)

o3d.t.io.write_point_cloud("water2.pcd", pcd)


### Pipe parameter detection

#### Generation of synethetic IFC element dataset

In [ ]:
#density = 1024
sample_size = 20
config_path = "config/pipeline.json"
blueprint = 'data/sample.ifc'

In [ ]:
"""
elbow - modelled as an IfcRevolvedAreaSolid 

params:
position - 3D coordinate
axis_direction - 3D vector, axis of revolution (z>=0)
axis_position - 3D coordinate
angle - angle of revolution (0 -> pi)
radius
"""


In [ ]:
"""
pipe - modelled as an IfcExtrudedAreaSolid

params:
position - 3D coordinate
extrusion_direction - 3D vector (z>=0)
length - 3D coordinate
radius
"""


In [ ]:
# create a new blank ifc file
def setup_ifc_file(blueprint):

    ifc = ifcopenshell.open(blueprint)
    ifcNew = ifcopenshell.file(schema=ifc.schema)
    
    owner_history = ifc.by_type("IfcOwnerHistory")[0]
    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    floor = ifc.by_type("IfcBuildingStorey")[0]
    
    ifcNew.add(project) 
    ifcNew.add(owner_history) 
    ifcNew.add(context) 
    ifcNew.add(floor)

    return ifcNew

In [ ]:
def createIfcPipe(r, l, d, p, ifc, ifc_info):
    cross_section = Circle_Section(r=r, ifcfile=ifc)
    green = ifc.createIfcColourRgb('green', Red=0.0, Green=0.9, Blue=0.0)

    beam = CreateBeam(ifc, container=ifc_info['floor'], name="pipe", 
                      section=cross_section, L=l, position=p,
                      direction=d, owner_history=ifc_info["owner_history"],
                      context=ifc_info["context"], colour=green)

In [ ]:
# generate a synthetic pipe
def create_pipe(config,  ifc, ifc_info):
    # generate parameters
    r = random.uniform(config['radius_range'][0], config['radius_range'][1])
    l = random.uniform(config['length_range'][0], config['length_range'][1])
    
    d = []
    for ax in config['extrusion_direction_range']:
        d.append(random.uniform(ax[0], ax[1]))
    d_np = np.array(d)
    d = (d_np/np.linalg.norm(d_np)).tolist()
    
    p = []
    for coord in config['coordinate_range']:
        p.append(random.uniform(coord[0], coord[1]))
    
    # center the element
    centerpoint = [(p[i] + (l*d[i])/2) for i in range(3)]
    p = [p[i] - centerpoint[i] for i in range(3)]
    print('c', p)
    
    print(r,l,d,p)
    
    createIfcPipe(r, l, d, p, ifc, ifc_info)
    metadata = {'radius':r, "direction":d, "length":l, "position":p}
    return metadata


In [ ]:
def synthetic_dataset(config, sample_size, element_class, output_base, blueprint):
    # setup
    f = open(config, 'r')
    config_data  = json.load(f)
    output_dir = os.path.join(output_base, element_class)
    #os.makedirs(output_dir)

    metadata = {}
    for i in range(sample_size):
        # generate ifc file
        ifc = setup_ifc_file(blueprint)
        owner_history = ifc.by_type("IfcOwnerHistory")[0]
        project = ifc.by_type("IfcProject")[0]
        context = ifc.by_type("IfcGeometricRepresentationContext")[0]
        floor = ifc.by_type("IfcBuildingStorey")[0]
        
        ifc_info = {"owner_history": owner_history,
            "project": project,
           "context": context, 
           "floor": floor}
        
        # generate ifc element
        if element_class == 'pipe':
            e = create_pipe(config_data[element_class], ifc, ifc_info)

        metadata[str(i)] = e
        ifc.write(os.path.join(output_dir, '%d.ifc' % i))
    
    with open(os.path.join(output_dir, 'metadata.json'), 'w') as f:
        json.dump(metadata, f)
        
        

In [ ]:
synthetic_dataset(config_path, sample_size, "pipe", 'output', blueprint)

In [ ]:
d = {}
ifcNew = setup_ifc_file('data/sample.ifc',d)
print(d)